# 🎨 Data Designer 101: Seeding Synthetic Data Generation with an External Dataset

#### 📚 What you'll learn

In this notebook, we will demonstrate how to seed synthetic data generation in Data Designer with an external dataset.

If this is your first time using Data Designer, we recommend starting with the [first notebook](/notebooks/1-the-basics/) in this 101 series.

### 📦 Import the essentials

- The `essentials` module provides quick access to the most commonly used objects.


In [1]:
from data_designer.essentials import (
    DataDesigner,
    DataDesignerConfigBuilder,
    InferenceParameters,
    ModelConfig,
    SeedConfig,
)

### ⚙️ Initialize the Data Designer interface

- `DataDesigner` is the main object that is used to interface with the library.


In [2]:
# TODO: fill path to blob storage to be able to access personas
BLOB_STORAGE_PATH = ""

data_designer_client = DataDesigner(blob_storage_path=BLOB_STORAGE_PATH)

### 🎛️ Define model configurations

- Each `ModelConfig` defines a model that can be used during the generation process.

- The "model alias" is used to reference the model in the Data Designer config (as we will see below).

- The "model provider" is the external service that hosts the model (see the [model config](/models/default-model-settings.md) docs for more details).

- By default, we use [build.nvidia.com](https://build.nvidia.com/models) as the model provider.


In [3]:
# This name is set in the model provider configuration.
MODEL_PROVIDER = "nvidia"

# The model ID is from build.nvidia.com.
MODEL_ID = "nvidia/nvidia-nemotron-nano-9b-v2"

# We choose this alias to be descriptive for our use case.
MODEL_ALIAS = "nemotron-nano-v2"

# This sets reasoning to False for the nemotron-nano-v2 model.
SYSTEM_PROMPT = "/no_think"

model_configs = [
    ModelConfig(
        alias=MODEL_ALIAS,
        model=MODEL_ID,
        provider=MODEL_PROVIDER,
        inference_parameters=InferenceParameters(
            temperature=0.5,
            top_p=1.0,
            max_tokens=1024,
        ),
    )
]

### 🏗️ Initialize the Data Designer Config Builder

- The Data Designer config defines the dataset schema and generation process.

- The config builder provides an intuitive interface for building this configuration.

- The list of model configs is provided to the builder at initialization.


In [4]:
config_builder = DataDesignerConfigBuilder(model_configs=model_configs)

## 🏥 Prepare a seed dataset

- For this notebook, we'll create a synthetic dataset of patient notes.

- We will _seed_ the generation process with a [symptom-to-diagnosis dataset](https://huggingface.co/datasets/gretelai/symptom_to_diagnosis).

- We already have the dataset downloaded in the [data](../data) directory of this repository.

<br>

> 🌱 **Why use a seed dataset?**
>
> - Seed datasets let you steer the generation process by providing context that is specific to your use case.
>
> - Seed datasets are also an excellent way to inject real-world diversity into your synthetic data.
>
> - During generation, prompt templates can reference any of the seed dataset fields.


In [5]:
# Download sample dataset from Github
import urllib.request

url = "https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/refs/heads/main/nemo/NeMo-Data-Designer/data/gretelai_symptom_to_diagnosis.csv"
local_filename, headers = urllib.request.urlretrieve(url, "gretelai_symptom_to_diagnosis.csv")

seed_dataset = SeedConfig(dataset=local_filename)

# Pass the reference to the config builder for use during generation.
config_builder.with_seed_dataset(seed_dataset)

DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
)

## 🎨 Designing our synthetic patient notes dataset

- Here we use `add_column` with keyword arguments (rather than imported config objects).

- Generally, we recommend using concrete objects, but this is a convenient shorthand.

- **Note**: The prompt template can reference fields from our seed dataset:
  - `{{ diagnosis }}` - the medical diagnosis from the seed data
  - `{{ patient_summary }}` - the symptom description from the seed data


In [6]:
config_builder.add_column(
    name="patient_sampler",
    column_type="sampler",
    sampler_type="person",
    drop=False,
)

config_builder.add_column(
    name="doctor_sampler",
    column_type="sampler",
    sampler_type="person",
    drop=False,
)

config_builder.add_column(
    name="patient_id",
    column_type="sampler",
    sampler_type="uuid",
    params={
        "prefix": "PT-",
        "short_form": True,
        "uppercase": True,
    },
)

config_builder.add_column(
    name="first_name",
    column_type="expression",
    expr="{{ patient_sampler.first_name}}",
)

config_builder.add_column(
    name="last_name",
    column_type="expression",
    expr="{{ patient_sampler.last_name }}",
)


config_builder.add_column(
    name="dob",
    column_type="expression",
    expr="{{ patient_sampler.birth_date }}",
)


config_builder.add_column(
    name="patient_email",
    column_type="expression",
    expr="{{ patient_sampler.email_address }}",
)


config_builder.add_column(
    name="symptom_onset_date",
    column_type="sampler",
    sampler_type="datetime",
    params={"start": "2024-01-01", "end": "2024-12-31"},
)

config_builder.add_column(
    name="date_of_visit",
    column_type="sampler",
    sampler_type="timedelta",
    params={"dt_min": 1, "dt_max": 30, "reference_column_name": "symptom_onset_date"},
)

config_builder.add_column(
    name="physician",
    column_type="expression",
    expr="Dr. {{ doctor_sampler.last_name }}",
)

config_builder.add_column(
    name="physician_notes",
    column_type="llm-text",
    prompt="""\
You are a primary-care physician who just had an appointment with {{ first_name }} {{ last_name }},
who has been struggling with symptoms from {{ diagnosis }} since {{ symptom_onset_date }}.
The date of today's visit is {{ date_of_visit }}.

{{ patient_summary }}

Write careful notes about your visit with {{ first_name }},
as Dr. {{ doctor_sampler.first_name }} {{ doctor_sampler.last_name }}.

Format the notes as a busy doctor might.
""",
    model_alias=MODEL_ALIAS,
    system_prompt=SYSTEM_PROMPT,
)

config_builder.validate()

[21:56:26] [INFO] ✅ Validation passed


DataDesignerConfigBuilder(
    seed_dataset: 'gretelai_symptom_to_diagnosis.csv'
    seed_dataset_columns: ['diagnosis', 'patient_summary']
    sampler_columns: [
        "patient_sampler",
        "doctor_sampler",
        "patient_id",
        "symptom_onset_date",
        "date_of_visit"
    ]
    llm_text_columns: ['physician_notes']
    expression_columns: [
        "first_name",
        "last_name",
        "dob",
        "patient_email",
        "physician"
    ]
)

### 🔁 Iteration is key – preview the dataset!

1. Use the `preview` method to generate a sample of records quickly.

2. Inspect the results for quality and format issues.

3. Adjust column configurations, prompts, or parameters as needed.

4. Re-run the preview until satisfied.


In [7]:
preview = data_designer_client.preview(config_builder)

[21:56:26] [INFO] 📸 Preview generation in progress
[21:56:26] [INFO] ✅ Validation passed
[21:56:26] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[21:56:26] [INFO] 🩺 Running health checks for models...
[21:56:26] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[21:56:28] [INFO]   |-- ✅ Passed!
[21:56:28] [INFO] 🌱 Sampling 10 records from seed dataset
[21:56:28] [INFO]   |-- seed dataset size: 820 records
[21:56:28] [INFO]   |-- sampling strategy: ordered
[21:56:28] [INFO] 🎲 Preparing samplers to generate 10 records across 5 columns
[21:56:28] [INFO] 🎲 👨‍💻 Initializing person generation
[21:56:43] [INFO] (💾 + 💾) Concatenating 2 datasets
[21:56:43] [INFO] 🧩 Generating column `first_name` from expression
[21:56:43] [INFO] 🧩 Generating column `last_name` from expression
[21:56:43] [INFO] 🧩 Generating column `dob` from expression
[21:56:43] [INFO] 🧩 Generating column `patient_email` from expression

In [8]:
# Run this cell multiple times to cycle through the 10 preview records.
preview.display_sample_record()

                                                   Seed Columns                                                    
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name            ┃ Value                                                                                         ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis       │ cervical spondylosis                                                                          │
├─────────────────┼───────────────────────────────────────────────────────────────────────────────────────────────┤
│ patient_summary │ I've been having a lot of pain in my neck and back. I've also been having trouble with my     │
│                 │ balance and coordination. I've been coughing a lot and my limbs feel weak.                    │
└─────────────────┴───────────────────────────────────────────────────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                 Generated Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name               ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler    │ {                                                                                          │
│                    │     'uuid': '06599092-3628-4ab2-959d-16b2d6b879a3',                                        │
│                    │     'first_name': 'Kisha',                                                                 │
│                    │     'middle_name': 'Mary',                                                                 │
│                    │     'last_name': 'Davisjames',                                                             │
│                    │     'sex': 'Female',                                                                       │
│                    │     'age': 35,                                                                             │
│                    │     'marital_status': 'never_married',                                                     │
│                    │     'street_name': 'Woolston St',                                                          │
│                    │     'street_number': 96,                                                                   │
│                    │     'unit': '',                                                                            │
│                    │     'postcode': '21223',                                                                   │
│                    │     'city': 'Baltimore',                                                                   │
│                    │     'district': 'Baltimore city',                                                          │
│                    │     'country': 'USA',                                                                      │
│                    │     'bachelors_field': None,                                                               │
│                    │     'education_level': 'associates',                                                       │
│                    │     'occupation': 'elementary_or_middle_school_teacher',                                   │
│                    │     'locale': 'en_US',                                                                     │
│                    │     'birth_date': '1990-07-22',  

In [9]:
# The preview dataset is available as a pandas DataFrame.
preview.dataset

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,patient_email,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,{'uuid': '06599092-3628-4ab2-959d-16b2d6b879a3...,{'uuid': '5bc83d7a-6e86-4f26-945f-6aab834c670d...,PT-D4F9BDF8,2024-08-22,2024-09-08,Kisha,Davisjames,1990-07-22,kisha.davisjames@icloud.com,Dr. Turner,"**Dr. Kyle Turner, MD** \n**Primary Care Phys..."
1,impetigo,I have a rash on my face that is getting worse...,{'uuid': '3a85f295-ebae-4842-a4e0-256440238444...,{'uuid': '28171b4d-c7a7-40a3-9a94-44a4cae523ed...,PT-5929721D,2024-12-25,2024-12-29,Nadine,Cammarata,1974-05-25,nadineacammarata@gmail.com,Dr. Estrada,"**Dr. Mabelle Estrada, Primary Care Physician*..."
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,{'uuid': '2da6e920-512b-4496-a333-70d0148f1f46...,{'uuid': '44ebc4a5-d803-4a01-a83c-aaf5a6fe7286...,PT-8D6D8B7C,2024-01-08,2024-01-15,Jennifer,Wasley,2005-10-26,jennifer.wasley26@gmail.com,Dr. Park,**Dr. Eden Park** \n**Primary Care Physician*...
3,arthritis,I have been having trouble with my muscles and...,{'uuid': '05aadd45-caba-4c5d-bab5-d0e9329d0c88...,{'uuid': '337f8191-06a8-4fee-832a-cbd4d334b643...,PT-0CF63045,2024-12-01,2024-12-29,Ivette,Turner,1945-06-14,ivette_turner@hotmail.com,Dr. Gardner,**Dr. Madeline Gardner** \n**Primary Care Phy...
4,dengue,I have been feeling really sick. My body hurts...,{'uuid': '16730a2b-1130-47e7-bac9-aa1364562462...,{'uuid': '338fe5b7-8202-44ac-98cb-3c9dbb91feda...,PT-B22177B2,2024-05-31,2024-06-07,Lloyd,George,1952-05-02,georgel83@gmail.com,Dr. Esparragoza,**Patient Name:** Lloyd George \n**Date of Vi...
5,common cold,I've been feeling really run down and weak. My...,{'uuid': '2f97fafa-8a72-4438-9ed5-859336ca9f3d...,{'uuid': '2966e96b-69be-405f-b69c-2d2d4b4d84ee...,PT-0D2CD12C,2024-12-16,2024-12-28,Leidi,Hernandez,1951-01-31,leidih@hotmail.com,Dr. Hammer,**Dr. Walter Hammer** \n**Primary Care Physic...
6,dengue,I have rashes all over my body. I also have a ...,{'uuid': '0db7bb75-9af3-4d22-ad6d-920a9ceb8845...,{'uuid': '3b83473e-5bc3-4c7a-aecc-472737560f4b...,PT-491A8EAF,2024-09-23,2024-09-28,Destiny,Howard,1977-04-04,destinyh77@hotmail.com,Dr. Bermudez,**Dr. John Bermudez** \n**Primary Care Physic...
7,dengue,I have been feeling nauseous and have a consta...,{'uuid': '6084aa6d-3f48-4ca7-93ee-04ddbd531c85...,{'uuid': '066af806-e3cb-4546-90d9-50bbf81f9797...,PT-D30BF5FF,2024-01-06,2024-02-01,Sarah,Swain,1991-09-05,sarahswain@yahoo.com,Dr. Delforge,**Dr. Joseph Delforge** \n**Primary Care Phys...
8,impetigo,I have a rash around my nose that is red and i...,{'uuid': '3c824360-438f-413c-973a-92afc16805f7...,{'uuid': '178cc823-adfe-4ee1-be9c-13466ab049fb...,PT-D3FB0F66,2024-02-08,2024-03-08,Daya,Lopera,1962-07-28,dayamarie1962@gmail.com,Dr. Holsclaw,"**Dr. Jessica Holsclaw, MD** \n**Primary Care..."
9,drug reaction,I have a rash on my chest and back and it itch...,{'uuid': '16504aa7-7ba1-4cfd-85a1-a0aa78800113...,{'uuid': '2d1bdfb5-969d-4cad-813c-4e842ced8d42...,PT-F0B6E70A,2024-08-23,2024-09-19,Janet,Madrid,2007-09-19,janet.madrid2007@icloud.com,Dr. Ferebee,"**Dr. Amy Ferebee, MD** \n**Primary Care Phys..."


### 📊 Analyze the generated data

- Data Designer automatically generates a basic statistical analysis of the generated data.

- This analysis is available via the `analysis` property of generation result objects.


In [10]:
# Print the analysis as a table.
preview.analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 10                              │ 13                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                      7 (70.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    10 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                     ┃         data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler                 │              dict │                        10 (100.0%) │               person │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ doctor_sampler                  │              dict │                        10 (100.0%) │               person │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ patient_id                      │            string │                        10 (100.0%) │                 uuid │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ symptom_onset_date              │            string │                        10 (100.0%) │             datetime │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ date_of_visit                   │            string │                          9 (90.0%) │            timedelta │
└─────────────────────────────────┴───────────────────┴────────────────────────────────────┴──────────────────────┘
                                                          

### 🆙 Scale up!

- Happy with your preview data?

- Use the `create` method to submit larger Data Designer generation jobs.


In [11]:
job_results = data_designer_client.create(config_builder, num_records=20)

[21:57:06] [INFO] 🎨 Creating Data Designer dataset
[21:57:06] [INFO] ✅ Validation passed
[21:57:06] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[21:57:06] [INFO] 🩺 Running health checks for models...
[21:57:06] [INFO]   |-- 👀 Checking 'nvidia/nvidia-nemotron-nano-9b-v2' in provider named 'nvidia' for model alias 'nemotron-nano-v2'...
[21:57:07] [INFO]   |-- ✅ Passed!
[21:57:07] [INFO] ⏳ Processing batch 1 of 1
[21:57:07] [INFO] 🌱 Sampling 20 records from seed dataset
[21:57:07] [INFO]   |-- seed dataset size: 820 records
[21:57:07] [INFO]   |-- sampling strategy: ordered
[21:57:07] [INFO] 🎲 Preparing samplers to generate 20 records across 5 columns
[21:57:07] [INFO] 🎲 🧑‍🎨 Initializing person generation
[21:57:22] [INFO] (💾 + 💾) Concatenating 2 datasets
[21:57:22] [INFO] 🧩 Generating column `first_name` from expression
[21:57:22] [INFO] 🧩 Generating column `last_name` from expression
[21:57:22] [INFO] 🧩 Generating column `dob` from expression
[21:57:22] [INFO] 🧩 Gener

In [12]:
# Load the generated dataset as a pandas DataFrame.
dataset = job_results.load_dataset()

dataset.head()

,diagnosis,patient_summary,patient_sampler,doctor_sampler,patient_id,symptom_onset_date,date_of_visit,first_name,last_name,dob,patient_email,physician,physician_notes
0,cervical spondylosis,I've been having a lot of pain in my neck and ...,"{'age': 60, 'bachelors_field': None, 'birth_da...","{'age': 59, 'bachelors_field': 'arts_humanitie...",PT-3515186E,2024-08-12,2024-08-25,Lucille,Kalam,1965-05-30,lucillemkalam65@hotmail.com,Dr. Hamilton,**Patient Name:** Lucille Kalam **Date of Vi...
1,impetigo,I have a rash on my face that is getting worse...,"{'age': 60, 'bachelors_field': 'arts_humanitie...","{'age': 78, 'bachelors_field': 'stem', 'birth_...",PT-5E9DD369,2024-02-21,2024-02-22,Stephen,Siler,1965-09-16,ssiler@hotmail.com,Dr. Goss,**Dr. Daniel Goss** **Primary Care Physician...
2,urinary tract infection,I have been urinating blood. I sometimes feel ...,"{'age': 55, 'bachelors_field': None, 'birth_da...","{'age': 55, 'bachelors_field': None, 'birth_da...",PT-CA40CDCF,2024-07-12,2024-07-16,Michael,Merrill,1970-08-14,michael.merrill@icloud.com,Dr. Allen,**Dr. Jeri Allen** **Primary Care Physician*...
3,arthritis,I have been having trouble with my muscles and...,"{'age': 40, 'bachelors_field': None, 'birth_da...","{'age': 26, 'bachelors_field': None, 'birth_da...",PT-78E18120,2024-01-13,2024-01-26,Stephanie,Johnston,1985-10-28,stephanie_johnston85@outlook.com,Dr. Mills,**Dr. Robert Mills** **Primary Care Physicia...
4,dengue,I have been feeling really sick. My body hurts...,"{'age': 42, 'bachelors_field': None, 'birth_da...","{'age': 42, 'bachelors_field': None, 'birth_da...",PT-952E25A7,2024-05-15,2024-06-08,William,Manthey,1983-05-11,williammanthey@hotmail.com,Dr. Lu,"**Dr. Min Lu, Primary Care Physician** **Pat..."


In [13]:
# Load the analysis results into memory.
analysis = job_results.load_analysis()

analysis.to_report()

──────────────────────────────────────── 🎨 Data Designer Dataset Profile ─────────────────────────────────────────

                                                                                                                   
                                                 Dataset Overview                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ number of records               ┃ number of columns               ┃ percent complete records                    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 20                              │ 13                              │ 100.0%                                      │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                              🌱 Seed-Dataset Columns                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                          ┃               data type ┃                           number unique values ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ diagnosis                            │                  string │                                     12 (60.0%) │
├──────────────────────────────────────┼─────────────────────────┼────────────────────────────────────────────────┤
│ patient_summary                      │                  string │                                    20 (100.0%) │
└──────────────────────────────────────┴─────────────────────────┴────────────────────────────────────────────────┘
                                                                                                                   
                                                                                                                   
                                                🎲 Sampler Columns                                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ column name                     ┃         data type ┃               number unique values ┃         sampler type ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ patient_sampler                 │              dict │                        20 (100.0%) │               person │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ doctor_sampler                  │              dict │                        20 (100.0%) │               person │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ patient_id                      │            string │                        20 (100.0%) │                 uuid │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ symptom_onset_date              │            string │                         19 (95.0%) │             datetime │
├─────────────────────────────────┼───────────────────┼────────────────────────────────────┼──────────────────────┤
│ date_of_visit                   │            string │                         19 (95.0%) │            timedelta │
└─────────────────────────────────┴───────────────────┴────────────────────────────────────┴──────────────────────┘
                                                          

## ⏭️ Next Steps

Use Data Designer to generate synthetic data for your specific use case!
